In [1]:
from TextClassifier.TfrecordGenerator import HanTfrecordGenerator, TextCnnTfrecordGenerator

In [2]:
data_path = 'data/trainSet/train_info_5w.csv'

In [3]:
htg = HanTfrecordGenerator(data_path)

load data...
Han start...


In [4]:
htg.preprocess_data(option=0)

/root/notebookPlace/wangxiao/TextClassifier/TfrecordGenerator.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.train_data.dropna(how='any', inplace=True)
/root/notebookPlace/wangxiao/TextClassifier/TfrecordGenerator.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.val_data.dropna(how='any', inplace=True)


load finished.
train/val/test = 940496/182623/203440


/root/notebookPlace/wangxiao/TextClassifier/TfrecordGenerator.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.test_data.dropna(how='any', inplace=True)


In [5]:
htg.out(path='data/trainSet')

processing...
train data
val data
vacab initial
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
fit
Instructions for updating:
Please use tensorflow/transform or tf.data.
fit train data


MemoryError: 

In [1]:
import numpy as np
import pandas as pd
import gc, time
from tensorflow.contrib import learn
import tensorflow as tf
from functools import reduce
import operator

In [2]:
data_path = 'data/trainSet/train_info_5w.csv'

In [3]:
df = pd.read_csv(data_path)

In [4]:
df

,rough_token,rigour_token,sent_num_rough,max_sentlen_rough,doc_len_rough,sent_num_rigour,max_sentlen_rigour,doc_len_rigour,cls,is_gen,train_val_test
0,2015 年 月 日 下午 时 左右 山马 公路 清泉 镇 西街 村 造纸厂 旁边 耕地 里...,时 山马 公路 清泉 镇 西街 村 造纸厂 耕地 转运 农家肥 张海 驾驶 拖拉机 耕地 时...,7,51,136,7,40,86,9001,0,3
1,2014 年 月 日 时 50 分 臧玉春 驾驶 黑 M5606E 福田 牌 普通 货车 兰...,时 分 臧玉春 驾驶 黑 M5606E 福田 牌 普通 货车 兰西县 临江镇 兴 河村 东井...,6,55,140,6,36,108,9001,0,2
2,陈天鹏 系 户口。陈天鹏 孙某 原均 系 星桥 中学 在校。高 福林 系 陈天鹏 父亲 汤妙...,陈天鹏 系 户口。陈天鹏 孙某 原均 系 星桥 中学 在校。高 福林 系 陈天鹏 父亲 汤妙...,11,58,232,11,39,157,9001,0,3
3,2014 年 月 日晚 21 时许 衡阳市 珠晖区 湖南路 邮政 宾馆 前 夜市 竞争 妻子...,时许 衡阳市 珠晖区 湖南路 邮政 宾馆 夜市 竞争 妻子 刘兰英 摊位 廖 发生争执 廖某...,2,41,54,2,31,42,9001,0,1
4,2010 年 月 13 日晚 朱 前进 放炮 时 朱毅超 右眼 炸伤。嗣后 朱毅超 之母 陪...,朱 前进 放炮 时 朱毅超 右眼 炸伤。朱毅超 之母 陪同 进入 解放军 第四军医大学 唐 ...,7,23,85,7,15,67,9001,0,1
5,2015 年 正月初三 上午 九 时 左右 蔡碧珍 张兰荣 发生 语言 冲突 发生 抓扯 张...,时 蔡碧珍 张兰荣 发生 语言 冲突 发生 抓扯 张兰荣 夫 刘友成 刘兰义 孙 云贵 杨道...,6,27,107,6,21,86,9001,0,1
6,2015 年 月 28 日 兴隆 汽校 朋友 检车 时 陪 检车 发生争执 厮打 打伤 住院...,兴隆 汽校 朋友 检车 时 陪 检车 发生争执 厮打 打伤 住院。受伤 兴隆 人民 医院 住...,4,16,46,4,11,33,9001,0,1
7,房院 位于 城 赵镇城 赵村 北 街门 位于 该院 东南角 东西向 街道 宽 米 南房正 一...,房院 位于 城 赵镇城 赵村 北 街门 位于 街道 宽 南房正 一村 街道 房院 处于 丁字...,7,29,107,7,23,86,9001,0,3
8,商学 明系 商 学文 堂兄 周 菊芳 夫 姚舜禹 表哥。双方 购买 原材料 问题 发生 纠纷...,商学 明系 商 学文 堂兄 周 菊芳 夫 姚舜禹 表哥。双方 购买 原材料 问题 发生 纠纷...,16,21,165,16,14,129,9001,0,1
9,2015 年 月 13 日 1000 00 元 丢失 当天 斯琴 去过 家 怀疑 斯琴 派出...,丢失 斯琴 去过 怀疑 斯琴 派出所 报 警。民警 找 双方 派出所 询问 完 双方 走出 ...,4,29,90,4,24,69,9001,0,2


In [6]:
df.dropna(how='any', inplace=True)

In [9]:
from collections import defaultdict

all_text = df['rigour_token'].tolist()
tmp = defaultdict(lambda : 0)

for text in all_text:
    for word in text.replace('。', ' ').split(' '):
        tmp[word] += 1

tmp = [(k, v) for k, v in tmp.items()]
tmp.sort(key=lambda x:x[1], reverse=True)

tmp = filter(lambda x: x[1] > 10, tmp)
tmp = [x[0] for x in tmp]

dic = dict()
for index, word in enumerate(tmp):
    dic[word] = index

In [10]:
dic

{'': 6374,
 '谢瑞煌': 174454,
 '周耀': 208601,
 '马某昂': 218194,
 '王长硕': 194244,
 '小云': 187733,
 '柳嘉镇': 150182,
 '李殿华': 142337,
 '李桂文': 149277,
 '罗宁合': 130272,
 '冯国庆': 78247,
 '分伍': 46993,
 '田螺': 163352,
 '陪': 3490,
 '杨燕燕': 109663,
 '含油': 192162,
 '上园': 120731,
 '绥劳人': 96289,
 '城子': 14401,
 '小汤': 163353,
 '李新戎': 183813,
 '杂货店': 25548,
 '唐绪军': 222971,
 '杨正才': 122878,
 '坠亡': 110133,
 '维诚': 97860,
 '庄恩军': 46789,
 '杜小芳': 200317,
 '许霞': 68906,
 '张卫明': 91513,
 '隆德': 18250,
 '坡坡': 108002,
 '韦世三': 193914,
 '袁晓庆': 86285,
 '郑仁': 136273,
 '李刘阳': 194245,
 '塑料门窗': 146107,
 '木匠': 22628,
 '周华平': 196237,
 '周俐群': 45808,
 '旧料': 62601,
 '忙': 16111,
 '陈卓立': 178960,
 '张艳敏': 93567,
 '黄爽': 226435,
 '刘建泉': 155791,
 '伊朗': 45728,
 '天生桥': 16854,
 '白瓷': 142341,
 '陆科': 93277,
 '深为': 190546,
 '陈爱萍': 47522,
 '兰洁': 226794,
 '潘启芝': 196238,
 '闫国斌': 222978,
 '宝平路': 177453,
 '英文翻译': 211032,
 'G308': 99789,
 '自然保护区': 53679,
 '彭永荣': 160732,
 '潘小庆': 222972,
 '彭康华': 181372,
 '路村': 52814,
 '方位': 20063,
 '陕化': 53533,
 '撞压': 162038,
 

In [11]:
def process_text(doc):
    res = []
    for sent in doc.split('。'):
        tmp = map(lambda x: dic[x] if x in dic else None, sent.split(' '))
        tmp = list(filter(lambda x: x is not None, tmp))
        if len(tmp) > 0:
            res.append(tmp)
    return res

In [12]:
res = df['rigour_token'].apply(process_text)
res

0          [[5, 478, 9309, 221, 5285, 167, 9813, 1710, 89...
1          [[5, 140, 13, 1960, 2882, 359, 144, 132, 51744...
2          [[18, 509], [1670, 28977, 18, 57818, 1682, 911...
3          [[271, 6173, 22071, 26080, 2888, 2111, 26036, ...
4          [[788, 4944, 21208, 5, 2423, 17449], [14956, 4...
5          [[5, 101322, 7, 8110, 4140, 7, 3998, 101322, 2...
6          [[2921, 109078, 1171, 38870, 5, 3490, 38870, 8...
7          [[13959, 150, 805, 21288, 887, 26984, 150, 450...
8          [[70691, 28553, 2510, 7510, 26085, 451, 11605,...
9          [[2772, 46138, 20557, 2732, 46138, 479, 1989, ...
10         [[5, 234, 153, 12944, 13, 11645, 25219, 3082, ...
11         [[98408, 2302, 13729, 46752, 45503, 42432, 187...
12         [[11, 18, 1381, 65], [5, 111655, 10049, 39928,...
13         [[9330, 27593, 14808, 2089, 2089, 2089, 145645...
14         [[271, 87755, 22696, 13329, 9987, 18390, 11242...
15         [[22574, 123219, 1356, 190], [271, 626, 2178, ...
16         [[18, 1055, 6

In [13]:
res.tolist()

[[[5,
   478,
   9309,
   221,
   5285,
   167,
   9813,
   1710,
   8911,
   125816,
   18095,
   13,
   1306,
   1710,
   24317,
   1306,
   959,
   68864,
   2962,
   1816,
   46037,
   526,
   5836,
   3,
   886,
   940,
   19668,
   82,
   9309,
   479,
   579,
   3,
   129222,
   479,
   5635,
   17,
   3,
   490,
   220],
  [19668, 64, 24, 241, 66, 2958, 1864, 79, 22],
  [19668,
   64,
   24,
   215828,
   226181,
   5040,
   84,
   7263,
   13636,
   9093,
   58457,
   62,
   8385],
  [926, 458, 4146, 84, 2958, 1864],
  [2958, 8385],
  [31, 62, 79, 426],
  [89, 51, 17, 25, 26, 205, 172, 385, 36, 222, 216, 169]],
 [[5,
   140,
   13,
   1960,
   2882,
   359,
   144,
   132,
   51744,
   41447,
   2230,
   6462,
   3621,
   36087,
   489,
   2277,
   98,
   215075,
   1885,
   54,
   948,
   29064,
   701,
   454,
   189,
   49,
   136,
   15],
  [2,
   51744,
   593,
   37,
   13,
   100,
   1795,
   8697,
   2752,
   2843,
   1220,
   475,
   2277,
   105,
   2,
   7,
   387,


In [6]:
data = df.loc[:, ['rough_token', 'cls', 'train_val_test']]
data.dropna(how='any', inplace=True)
len(data)

1326559

In [7]:
train_data = data[data['train_val_test'] == 1]
val_data = data[data['train_val_test'] == 2]
test_data = data[data['train_val_test'] == 3]

In [8]:
def helper(doc):
    sents = map(lambda x: x.strip(), doc.split('。'))
    sents = filter(lambda x: len(x) > 0, sents)
    return list(sents)

In [9]:
# data['src'] = data['rough_token'].apply(helper)
# data

In [10]:
train_x_text = list(map(lambda x: helper(x), train_data['rough_token'].tolist()))
train_y = train_data['cls'].tolist()

code2label = {}
for index, item in enumerate(list(set(train_y))):
    code2label[item] = index
y_train = np.array(list(map(lambda x: code2label[x], train_y)))

# del train_data
# gc.collect()

In [11]:
train_x_text[0]

['2014 年 月 日晚 21 时许 衡阳市 珠晖区 湖南路 邮政 宾馆 前 夜市 竞争 妻子 刘兰英 摊位 廖 发生争执 之后 廖某 甲 丈夫 谢某 甲 廖 丈夫 谢 加入 张 见状 加入 争吵 厮打 开来 双方 发生 群 造成 张 受伤',
 '事后 衡阳市 第一 人民 医院 门诊 检查 治疗 认为 生命 健康权 侵害 诉至']

In [12]:
val_x_text = list(map(lambda x: helper(x), val_data['rough_token'].tolist()))
y_val = np.array(list(map(lambda x: code2label[x], val_data['cls'].tolist())))

In [13]:
val_x_text[0]

['2014 年 月 日 时 50 分 臧玉春 驾驶 黑 M5606E 福田 牌 普通 货车 兰西县 临江镇 兴 河村 东井子 屯 内村 道 由东向西 倒车 过程 中 屯西 交叉口 处 行人 裴延贵 张桂芳 丈夫 裴俊国 裴俊权 裴俊艳 父亲 相撞 造成 裴延贵 死亡 交通事故',
 '本次 事故 兰西县 交警大队 认定 臧玉春 驾驶 机动车 观察 瞭望 不够 采取措施 不当 违反 倒车 规定 事故 发生 原因 负 事故 责任 裴延贵',
 '查明 裴延贵 农村户口',
 '臧玉春 驾驶 肇事 车辆 阳光 农业 相互 保险公司 绥化 中心 支公司 投保 交强险 期限 2013 年 12 月 日 起至 2014 年 12 月 日 止 双方 保单 中 责任 限额 进行 规定 死亡 伤残 赔偿 限额 110 000 00 元 医疗 费用 赔偿 限额 10 000 00 元 财产 损失赔偿 限额 000 00 元',
 '开庭审理 前 臧玉春 自行 和解 申请 撤回 臧玉春 起诉',
 '裴延贵 长女 裴俊英 声明 放弃 赔偿金 请求']

In [18]:
val_all_sent = []
i = 0
for doc in val_x_text:
    val_all_sent.extend(doc)
len(val_all_sent)

1295129

In [19]:
train_all_sent = []
i = 0
for doc in train_x_text:
    train_all_sent.extend(doc)
len(train_all_sent)

6510358

In [20]:
tmp = []
tmp.extend(val_all_sent)
tmp.extend(train_all_sent)
len(tmp)

7805487

In [20]:
tmp.shape

(1123119,)

In [21]:
tmp[0]

['2014 年 月 日晚 21 时许 衡阳市 珠晖区 湖南路 邮政 宾馆 前 夜市 竞争 妻子 刘兰英 摊位 廖 发生争执 之后 廖某 甲 丈夫 谢某 甲 廖 丈夫 谢 加入 张 见状 加入 争吵 厮打 开来 双方 发生 群 造成 张 受伤',
 '事后 衡阳市 第一 人民 医院 门诊 检查 治疗 认为 生命 健康权 侵害 诉至']

In [22]:
z = np.array([[1, 2, 3, 4],[5, 6, 7, 8],[9, 10, 11, 12],[13, 14, 15, 16]])

In [23]:
z

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [24]:
z.reshape(-1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [7]:
import operator
from functools import reduce
a = [['I love you', 'This must be boy', 'This is a a dog', 'abc a'],
          ['This is a cat','hao high ou', 'wo ai bei jing tian an men']]
reduce(operator.add, a)


['I love you',
 'This must be boy',
 'This is a a dog',
 'abc a',
 'This is a cat',
 'hao high ou',
 'wo ai bei jing tian an men']

In [2]:
from collections import defaultdict

In [23]:
all_text = ['I love you', 'This must be boy', 'This is a a dog', 'abc a',
          'This is a cat','hao high ou', 'wo ai bei jing tian an men']
dic = defaultdict(lambda : 0)
for text in all_text:
    for word in text.replace('。', ' ').split(' '):
        dic[word] += 1

In [24]:
dic = [(k, v) for k,v in dic.items()]

In [25]:
dic.sort(key=lambda x:x[1], reverse=True)
dic = filter(lambda x: x[1] > 1, dic)
dic = [x[0] for x in dic]

In [26]:
dic

['a', 'This', 'is']

In [29]:
res = dict()
for index,word in enumerate(dic):
    res[word] = index
res

{'This': 1, 'a': 0, 'is': 2}